<a href="https://colab.research.google.com/github/sholkojr/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRe

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-14 01:57:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.46MB/s    in 0.2s    

2020-12-14 01:57:07 (5.46 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

### Recreate the vine DataFrame

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
from pyspark.sql.types import IntegerType

In [8]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).withColumn("star_rating", df["star_rating"].cast(IntegerType()))

vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [9]:
vine_df.count()

4850360

### Create tables from the Vine DataFrame per Deliverable 2 Instructions   

In [12]:
# Filter for reviews with a total vote count greater than or equal to 20
vine_min_20_df = vine_df.filter("total_votes >= 20")

vine_min_20_df.count()

67855

In [13]:
# Filter for reviews with at least 50% helpful votes
vine_helpful_df = vine_min_20_df.filter("helpful_votes / total_votes >= 0.5")

vine_helpful_df.count()

61948

In [18]:
# Filter for paid reviewers
vine_paid_df = vine_helpful_df.filter("vine == 'Y'")

vine_paid_df.show(5)
vine_paid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



334

In [22]:
# Filter for unpaid reviewers
vine_unpaid_df = vine_helpful_df.filter("vine == 'N'")

vine_unpaid_df.show(5)
vine_unpaid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



61614

In [23]:
# Find Total Reviews
total_reviews = vine_helpful_df.count()
total_reviews

61948

In [24]:
# Find Total Paid 5 Star Reviews
paid_5star = vine_paid_df.filter("star_rating == 5").count()
paid_5star

139

In [25]:
# Find Total Unpaid 5 Star Reviews
unpaid_5star = vine_unpaid_df.filter("star_rating == 5").count()
unpaid_5star

32665

In [29]:
# Find Total Paid 5 Star Percentage
paid_5star_pct = paid_5star / vine_paid_df.count() * 100
paid_5star_pct

41.61676646706587

In [30]:
# Find Total Unpaid 5 Star Percentage
unpaid_5star_pct = unpaid_5star / vine_unpaid_df.count() * 100
unpaid_5star_pct

53.01554841432142